# Retrieving XMLs from eSc
We retrieve xmls of document parts from eScriptorium,
in order to prepare them for processing with Passim.

# Notebook initialization

In [1]:
from functions import *
from packages import *

switching to  msIA


Get the document parts xmls from eScriptorium

In [2]:
doc_pk = 4077
get_basic_info(doc_pk)

get document segmentation ontology for document:  4077
https://msia.escriptorium.fr/api/documents/4077/
Document: 4077  with  901  parts
region types: [{'pk': 6338, 'name': 'Catchword'}, {'pk': 3, 'name': 'Commentary'}, {'pk': 6345, 'name': 'FooterCentral'}, {'pk': 6343, 'name': 'FooterLeft'}, {'pk': 6342, 'name': 'FooterRight'}, {'pk': 6341, 'name': 'Handwritten'}, {'pk': 4, 'name': 'Illustration'}, {'pk': 2, 'name': 'Main'}, {'pk': 6339, 'name': 'MainCentral'}, {'pk': 6337, 'name': 'MainLeft'}, {'pk': 6336, 'name': 'MainRight'}, {'pk': 6340, 'name': 'Margin'}, {'pk': 6335, 'name': 'RunningHeader'}, {'pk': 6344, 'name': 'Stamp'}, {'pk': 1, 'name': 'Title'}]
line types: [{'pk': 14596, 'name': 'Handwritten'}, {'pk': 14592, 'name': 'MainCentral'}, {'pk': 14594, 'name': 'MainLeft'}, {'pk': 14593, 'name': 'MainRight'}, {'pk': 14595, 'name': 'NotMain'}]
transcription_level_list: [{'pk': 11018, 'name': 'MT_NoVoc_concatenated_pip_test_regexp_14/05/2024-14_39_17', 'archived': False, 'avg_confi

(901,
 [{'pk': 11018,
   'name': 'MT_NoVoc_concatenated_pip_test_regexp_14/05/2024-14_39_17',
   'archived': False,
   'avg_confidence': 0.9743265139608499},
  {'pk': 11020,
   'name': 'Machzor_Rosh_Hashanah_Ashkenaz_clean_concatenated_pip_test_regexp_14/05/2024-14_39_17',
   'archived': False,
   'avg_confidence': 0.9852161475573408},
  {'pk': 11021,
   'name': 'Siddur_Ashkenaz_clean_concatenated_pip_test_regexp_14/05/2024-14_39_17',
   'archived': False,
   'avg_confidence': 0.9817352012461885},
  {'pk': 11022,
   'name': 'Siddur_Ashkenaz_novoc_no_lbs_Daniel_pip_test_regexp_14/05/2024-14_39_17',
   'archived': False,
   'avg_confidence': 0.9881690782281187},
  {'pk': 11019,
   'name': 'Machzor_Yom_Kippur_Ashkenaz_clean_concatenated_pip_test_regexp_14/05/2024-14_39_17',
   'archived': False,
   'avg_confidence': 0.9881690782281187},
  {'pk': 10909,
   'name': 'Machzor_Yom_Kippur_Ashkenaz_clean_concatenated_pip_test_regexp_03/05/2024-10_37_23',
   'archived': True,
   'avg_confidence':

In [3]:
# get the region type list
region_type_list, line_type_list = get_document_segmentation_ontology(doc_pk,print_status=False)
print(region_type_list)


# choose the region type pk
# We want the regions {'pk': 6337, 'name': 'MainLeft'}, {'pk': 6336, 'name': 'MainRight'}
region_type_pk_list = [6337, 6336]
print(region_type_pk_list)

# Choose the transcription level
tr_level_pk = 9996

[{'pk': 6338, 'name': 'Catchword'}, {'pk': 3, 'name': 'Commentary'}, {'pk': 6345, 'name': 'FooterCentral'}, {'pk': 6343, 'name': 'FooterLeft'}, {'pk': 6342, 'name': 'FooterRight'}, {'pk': 6341, 'name': 'Handwritten'}, {'pk': 4, 'name': 'Illustration'}, {'pk': 2, 'name': 'Main'}, {'pk': 6339, 'name': 'MainCentral'}, {'pk': 6337, 'name': 'MainLeft'}, {'pk': 6336, 'name': 'MainRight'}, {'pk': 6340, 'name': 'Margin'}, {'pk': 6335, 'name': 'RunningHeader'}, {'pk': 6344, 'name': 'Stamp'}, {'pk': 1, 'name': 'Title'}]
[6337, 6336]


In [4]:
# get all the parts from the document
all_parts = get_all_parts(doc_pk)

# get the part pk list
part_pk_list = [ part['pk'] for part in all_parts ]

print(f"{len(part_pk_list)} parts found")

901 parts found


In [5]:


# get the xmls of the parts from eScriptorium
export_xml(doc_pk,part_pk_list,tr_level_pk,region_type_pk_list,include_undefined = False, include_orphan = False, file_format = 'alto',include_images = False, print_status = True)


200
b'{"status":"ok"}'


<Response [200]>

--------
